In [9]:
import pandas as pd
import re

RAW_PATH = "../data/raw/international_bestsellers.csv"
OUT_PATH = "../data/derived/domestic_share_top10_marketgroup_yearly_2013_2022.csv"

YEAR_MIN, YEAR_MAX = 2013, 2022
MARKETS = ["Germany", "United States", "France", "Italy", "Spain"]  # EU peers are computed from these

df = pd.read_csv(RAW_PATH)

# --- build year from date (dataset has 'date', not 'year') ---
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["year"] = df["date"].dt.year

# --- clean rank ---
df["rank"] = pd.to_numeric(df["rank"], errors="coerce")

# --- filter to scope ---
df = df[df["year"].between(YEAR_MIN, YEAR_MAX)]
df = df[df["country"].isin(MARKETS)]
df = df[df["rank"].between(1, 10)]

# --- domestic definition (robust):
# domestic if market country appears in the nationality string as a token
# Handles: "United States; Australia", "United States; United States", "Australia, United States"
def is_domestic(nationality, market):
    if pd.isna(nationality) or pd.isna(market):
        return 0
    # split on common separators ; and , then strip spaces
    parts = [p.strip() for p in re.split(r"[;,]", str(nationality)) if p.strip()]
    return 1 if market in parts else 0

df["is_domestic"] = [
    is_domestic(nat, ctry) for nat, ctry in zip(df["nationality"], df["country"])
]

# --- per (year, country) ---
by_country = (
    df.groupby(["year", "country"], as_index=False)
      .agg(
          domestic_share_top10=("is_domestic", "mean"),
          n_top10_valid=("is_domestic", "size"),
          n_domestic=("is_domestic", "sum"),
      )
)

# --- build market groups ---
germany = by_country[by_country["country"] == "Germany"].copy()
germany["market_group"] = "Germany"

us = by_country[by_country["country"] == "United States"].copy()
us["market_group"] = "United States"

eu = by_country[by_country["country"].isin(["France", "Italy", "Spain"])].copy()

# EU peers share = unweighted mean of the *country-level shares* (NOT pooled titles)
eu_peers = (
    eu.groupby("year", as_index=False)
      .agg(
          domestic_share_top10=("domestic_share_top10", "mean"),
          # counts below are pooled totals for transparency only (share is NOT computed from them)
          n_top10_valid=("n_top10_valid", "sum"),
          n_domestic=("n_domestic", "sum"),
      )
)
eu_peers["market_group"] = "EU peers (avg)"

final = pd.concat(
    [germany[["year","market_group","domestic_share_top10","n_top10_valid","n_domestic"]],
     eu_peers[["year","market_group","domestic_share_top10","n_top10_valid","n_domestic"]],
     us[["year","market_group","domestic_share_top10","n_top10_valid","n_domestic"]]],
    ignore_index=True
)

# --- convert to percent + rounding (as requested) ---
final["domestic_share_top10_pct"] = (final["domestic_share_top10"] * 100).round(1)

# keep tidy order
final = final.sort_values(["year", "market_group"]).reset_index(drop=True)

# output schema exactly as the viz expects
final_out = final[["year", "market_group", "domestic_share_top10_pct", "n_top10_valid", "n_domestic"]]
final_out.to_csv(OUT_PATH, index=False)

display(final_out.head(12))
print(f"Wrote: {OUT_PATH}")

,year,market_group,domestic_share_top10_pct,n_top10_valid,n_domestic
0,2013,EU peers (avg),59.9,200,122
1,2013,Germany,28.6,70,20
2,2013,United States,84.3,70,59
3,2014,EU peers (avg),67.8,360,244
4,2014,Germany,33.3,120,40
5,2014,United States,85.8,120,103
6,2015,EU peers (avg),70.3,360,253
7,2015,Germany,34.2,120,41
8,2015,United States,84.2,120,101
9,2016,EU peers (avg),72.7,359,261


Wrote: ../data/derived/domestic_share_top10_marketgroup_yearly_2013_2022.csv
